In [2]:
import h2o
import os
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

# Define the path to the file
file_path = "C:/Users/Utilizador/Desktop/Cenas_do_Instituto/Ano3/IA/IA2324SlaughterSquad/SlaughterSquad/src/main/java/com/slaughtersquad/datasets/dataset.csv"

# Import the file
raw_data = h2o.import_file(path = [file_path], parse = False)

parse_setup = h2o.parse_setup(
    raw_frames = raw_data, 
    destination_frame = "parsed_data", 
    separator = ";",
    column_names = ["currentPositionX", "currentPositionY", "distance", "velocity", "bearing", "futureBearing", "enemyPositionX", "enemyPositionY", "predictedEnemyPositionX", "predictedEnemyPositionY", "gunTurnRemaining", "gunHeat", "hitOrNot"],
    column_types = ["numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "numeric", "enum"],
    header = 1,
)

# Parse the file with the specified setup options
df = h2o.parse_raw(parse_setup)

# Display the dataframe
df.head()

# Define features (or predictors) and target (or response)
features = [
    "currentPositionX", "currentPositionY", "distance", 
    "velocity", "bearing", "futureBearing", 
    "enemyPositionX", "enemyPositionY", 
    "predictedEnemyPositionX", "predictedEnemyPositionY", 
    "gunTurnRemaining", "gunHeat"
]

target = "hitOrNot" # Binary target column

# Ensure the target is binary (factor)
df[target] = df[target].asfactor()

# Split the data into training, validation, and test sets
train, valid, test = df.split_frame(
    ratios = [0.7, 0.15],
    seed = 123
)

# Train the model
model = H2ORandomForestEstimator(ntrees = 50, max_depth = 25, min_rows = 10)
model.train(x = features, y = target, training_frame = train, validation_frame = valid)

# Get performance metrics on the validation set
validation_metrics = model.model_performance(valid)
print("Validation Metrics:")
print(validation_metrics)

# Get performance metrics on the test set
test_metrics = model.model_performance(test)
print("Test Metrics:")
print(test_metrics)

# AUC
auc = validation_metrics.auc()
print("Validation AUC:", auc)

# Logloss
logloss = validation_metrics.logloss()
print("Validation Logloss:", logloss)

# Confusion Matrix
confusion_matrix = test_metrics.confusion_matrix()
print("Test Confusion Matrix:")
print(confusion_matrix)

file_path = "C:/Users/Utilizador/Desktop/Cenas_do_Instituto/Ano3/IA/IA2324SlaughterSquad/SlaughterSquad/target/classes/com/slaughtersquad/sampleRobots/IntelligentRobot_v2.data"

# Save the model
mojo_path = model.download_mojo(path = file_path, get_genmodel_jar = True)

# Get the model type --> Regression, Binomial, Multinomial, Clustering, AutoEncoder, DimReduction, WordEmbedding
model_category = model.type

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 mins 25 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 16 days
H2O_cluster_name:,H2O_from_python_Utilizador_elcclj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.591 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Validation Metrics:
ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.17605427822279282
RMSE: 0.4195882245997769
LogLoss: 0.5259427459249735
Mean Per-Class Error: 0.20300879141674422
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
hit    hitOrNot    no_hit    Error     Rate
-----  ----------  --------  --------  ---------------
49103  0           7210      0.128034  7 210 / 56 313
0      0           0         nan       0 / 0
15322